In [ ]:
pip install qiskit


In [9]:

# Swap test fidelity comparison
# 7.11.20 B Manifold
# fredkins gates (7): https://pdfs.semanticscholar.org/4774/7792a7b13028e47c9daa2259f77d264a27a8.pdf
# Noise model is used atm, access is limited!

from qiskit import QuantumCircuit, Aer, execute, IBMQ
import qiskit.providers.aer.noise as noise
from qiskit.extensions import UnitaryGate
import numpy as np
import time

#TOKEN = 'd9f80fb1b7e356d27ee73f85b540021cb0ff9e92675bd1c12e28b734f75ba380f0757b2c69b4bb6a402f2694f739e5de1c4be7a938b7e0bcaf684a22dfcd2884'
#IBMQ.enable_account(TOKEN)
provider = IBMQ.get_provider()
simulator = Aer.get_backend('qasm_simulator')


# Set noise model
backend = 'ibmq_16_melbourne'
device = provider.get_backend(backend)
properties = device.properties()
gate_lengths = noise.device.parameters.gate_length_values(properties)
noise_model = noise.device.basic_device_noise_model(properties, gate_lengths=gate_lengths)
basis_gates = noise_model.basis_gates
coupling_map = device.configuration().coupling_map


def state_creation_circuit(index, qubits):
    # since fredkins gates are used to find fidelity, modded to include both registers
    # anc is 0

    if index == 1:
        state_create = QuantumCircuit(2 * qubits + 1, 1)
        return state_create
    elif index ==2:
        state_create = QuantumCircuit(2 * qubits + 1, 1)
        state_create.x(1)
        state_create.x(1 + qubits)
        return state_create
    elif index == 3:
        state_create = QuantumCircuit(2 * qubits + 1, 1)
        state_create.x(qubits)
        state_create.x(qubits + qubits)
        state_create.x(1)
        state_create.x(1 + qubits)
        return state_create
    elif index == 4:
        state_create=QuantumCircuit(2 * qubits + 1, 1)
        for i in range(1, qubits + 1):
            state_create.x(i)
            state_create.x(i + qubits)
        return state_create


def run_job(qc, compare):

    shots = 1024
    prob = 0

    if compare == 'Ideal':
        sim = Aer.get_backend('qasm_simulator')
        result = execute(qc, sim, shots=shots).result()
        counts = [result.get_counts(i) for i in range(len(result.results))]
        if '0' in counts[0].keys():
            prob += counts[0]['0'] / shots
        return prob

    elif compare == 'Backend':
        device = provider.get_backend(backend)
        result = execute(qc, backend=simulator, shots=shots,noise_model=noise_model,
                             basis_gates=basis_gates, coupling_map=coupling_map).result()
        counts = [result.get_counts(i) for i in range(len(result.results))]
        if '0' in counts[0].keys():
            prob += counts[0]['0'] / shots
        return prob


def v_matrix(dagger):

    w = (1 + 1j) / 2
    v = np.array([[w, -1j * w],[-1j * w, w]])
    if dagger:
        return np.conj(v).T
    else:
        return v


def fredkin7(qc, control, t1, t2):

    # assumes t1 < t2 since ancilla is q0
    v = UnitaryGate(v_matrix(False)).control()
    v_dagger = UnitaryGate(v_matrix(True)).control()
    qc.cx(control, t1)
    qc.append(v, [t1, t2])
    qc.append(v, [0, t2])
    qc.cx(control, t1)
    qc.append(v_dagger, [t1, t2])
    qc.cx(control, t1)
    qc.cx(t2, t1)

    return


def fredkin3(qc, control, t1, t2):
    qc.cx(t2, t1) 
    qc.ccx(control, t1, t2) 
    qc.cx(t2, t1) 
    return


def createOneState(qc, qubits): # has been fixed for registers
    qc.x(1)
    qc.x(1 + qubits)


def teleport_circuit(i, fredkin_alg, compare): # has been fixed for registers

    qc = state_creation_circuit(i, 3)
    qubits = 3
    createOneState(qc, 3)
    qc.h(2)
    qc.h(2 + qubits)
    qc.cx(2, 3)
    qc.cx(2 + qubits, 3 + qubits)
    qc.cx(1, 2)
    qc.cx(1 + qubits, 2 + qubits)
    qc.h(1)
    qc.h(1 + qubits)
    qc.cx(2, 3)
    qc.cx(2 + qubits, 3 + qubits)
    qc.cz(1, 3)
    qc.cz(1 + qubits, 3 + qubits)
    for x in range(1, qubits + 1):
        fredkin_alg(qc, 0, x, x + qubits)
    qc.measure(0, 0)
    fid = run_job(qc, compare)
    return fid


def w_state_circuit(i, fredkin_alg, compare): # has been fixed for registers

    w3 = state_creation_circuit(i, 3)
    qubits = 3
    w3.ry(-1.23096, 1)
    w3.ry(-1.23096, 1 + qubits)
    w3.x(2)
    w3.x(2 + qubits)
    w3.x(3)
    w3.x(3 + qubits)
    w3.cx(1, 2)
    w3.cx(1 + qubits, 2 + qubits)
    w3.ry(np.pi / 4, 1)
    w3.ry(np.pi / 4, 1 + qubits)
    w3.cx(2, 1)
    w3.cx(2 + qubits, 1 + qubits)
    w3.ry(-1 * np.pi / 4, 1)
    w3.ry(-1 * np.pi / 4, 1 + qubits)
    w3.x(1)
    w3.x(1 + qubits)
    w3.x(2)
    w3.x(2 + qubits)
    w3.cx(1, 3)
    w3.cx(1 + qubits, 3 + qubits)
    w3.cx(2, 3)
    w3.cx(2 + qubits, 3 + qubits)
    for x in range(1, qubits + 1):
        fredkin_alg(w3, 0, x, x + qubits)
    w3.measure(0, 0)
    fid = run_job(w3, compare)

    return fid


def ghz_circuit(i, fredkin_alg, compare): # has been fixed for registers

    qc = state_creation_circuit(i, 4)
    qubits = 4
    qc.h(1)
    qc.h(1 + qubits)
    qc.cx(1, 2)
    qc.cx(1 + qubits, 2 + qubits)
    qc.cx(2, 3)
    qc.cx(2 + qubits, 3 + qubits)
    qc.cx(3, 4)
    qc.cx(3 + qubits, 4 + qubits)
    for x in range(1, qubits + 1):
        fredkin_alg(qc, 0, x, x + qubits)
    qc.measure(0, 0)
    fid = run_job(qc, compare)

    return fid




# Test Fredkins Gates!

circuit_algs = [teleport_circuit, w_state_circuit, ghz_circuit]
circuit_tags = ['Teleport', 'W-State', 'GHZ']
fredkin_algs = [fredkin7, fredkin3]
fredkin_tags = ['Fredkin7', 'Fredkin3']

alg_dex = 0
fredkin_7, fredkin_3 = [], []
fredkins = [fredkin_7, fredkin_3]

print('Testing Teleport...')
print('__________')
alg_dex = 0 
alg = circuit_algs[0]
for choice in range(1, 5):
    for fredkin_choice in range(2):
        # compare ideal sim to backend result
        comps = []
        for compare in ['Ideal', 'Backend']: 
            fredkin = fredkin_algs[fredkin_choice]
            fidelity = alg(choice, fredkin, compare)
            print(compare, 'fidelity for', circuit_tags[alg_dex], 'with', fredkin_tags[fredkin_choice],
                    'and initial state', choice, ':', fidelity)
            comps.append(fidelity)
        fredkins[fredkin_choice].append(comps[0] - comps[1])
print('__________')

print('Calculating average differences... ')
avg3 = sum(fredkin_3) / len(fredkin_3)
avg7 = sum(fredkin_7) / len(fredkin_7)
print('Fredkin7: ', avg7)
print('Fredkin3: ', avg3)



Testing Teleport...
__________
Ideal fidelity for Teleport with Fredkin7 and initial state 1 : 1.0
Backend fidelity for Teleport with Fredkin7 and initial state 1 : 0.7353515625
Ideal fidelity for Teleport with Fredkin3 and initial state 1 : 1.0
Backend fidelity for Teleport with Fredkin3 and initial state 1 : 0.734375
Ideal fidelity for Teleport with Fredkin7 and initial state 2 : 1.0
Backend fidelity for Teleport with Fredkin7 and initial state 2 : 0.7158203125
Ideal fidelity for Teleport with Fredkin3 and initial state 2 : 1.0
Backend fidelity for Teleport with Fredkin3 and initial state 2 : 0.7353515625
Ideal fidelity for Teleport with Fredkin7 and initial state 3 : 1.0
Backend fidelity for Teleport with Fredkin7 and initial state 3 : 0.841796875
Ideal fidelity for Teleport with Fredkin3 and initial state 3 : 1.0
Backend fidelity for Teleport with Fredkin3 and initial state 3 : 0.6875
Ideal fidelity for Teleport with Fredkin7 and initial state 4 : 1.0
Backend fidelity for Teleport 

In [11]:
backend = 'ibmq_16_melbourne'
circuit_algs = [teleport_circuit, w_state_circuit, ghz_circuit]
circuit_tags = ['Teleport', 'W-State', 'GHZ']
fredkin_algs = [fredkin7, fredkin3]
fredkin_tags = ['Fredkin7', 'Fredkin3']

alg_dex = 0
fredkin_7, fredkin_3 = [], []
fredkins = [fredkin_7, fredkin_3]

print('Testing W-State...')
print('__________')
alg_dex = 1
alg = circuit_algs[1]
for choice in range(1, 5):
    for fredkin_choice in range(2):
        # compare ideal sim to backend result
        comps = []
        for compare in ['Ideal', 'Backend']: 
            fredkin = fredkin_algs[fredkin_choice]
            fidelity = alg(choice, fredkin, compare)
            print(compare, 'fidelity for', circuit_tags[alg_dex], 'with', fredkin_tags[fredkin_choice],
                    'and initial state', choice, ':', fidelity)
            comps.append(fidelity)
        fredkins[fredkin_choice].append(comps[0] - comps[1])
print('__________')

print('Calculating average differences... ')
avg3 = sum(fredkin_3) / len(fredkin_3)
avg7 = sum(fredkin_7) / len(fredkin_7)
print('Fredkin7: ', avg7)
print('Fredkin3: ', avg3)

Testing W-State...
__________
Ideal fidelity for W-State with Fredkin7 and initial state 1 : 1.0
Backend fidelity for W-State with Fredkin7 and initial state 1 : 0.7060546875
Ideal fidelity for W-State with Fredkin3 and initial state 1 : 1.0
Backend fidelity for W-State with Fredkin3 and initial state 1 : 0.701171875
Ideal fidelity for W-State with Fredkin7 and initial state 2 : 1.0
Backend fidelity for W-State with Fredkin7 and initial state 2 : 0.7333984375
Ideal fidelity for W-State with Fredkin3 and initial state 2 : 1.0
Backend fidelity for W-State with Fredkin3 and initial state 2 : 0.8037109375
Ideal fidelity for W-State with Fredkin7 and initial state 3 : 1.0
Backend fidelity for W-State with Fredkin7 and initial state 3 : 0.6982421875
Ideal fidelity for W-State with Fredkin3 and initial state 3 : 1.0
Backend fidelity for W-State with Fredkin3 and initial state 3 : 0.7626953125
Ideal fidelity for W-State with Fredkin7 and initial state 4 : 1.0
Backend fidelity for W-State with 

In [12]:
backend = 'ibmq_16_melbourne'
circuit_algs = [teleport_circuit, w_state_circuit, ghz_circuit]
circuit_tags = ['Teleport', 'W-State', 'GHZ']
fredkin_algs = [fredkin7, fredkin3]
fredkin_tags = ['Fredkin7', 'Fredkin3']

alg_dex = 0
fredkin_7, fredkin_3 = [], []
fredkins = [fredkin_7, fredkin_3]

print('Testing GHZ ...')
print('__________')
alg_dex = 2
alg = circuit_algs[2]
for choice in range(1, 5):
    for fredkin_choice in range(2):
        # compare ideal sim to backend result
        comps = []
        for compare in ['Ideal', 'Backend']: 
            fredkin = fredkin_algs[fredkin_choice]
            fidelity = alg(choice, fredkin, compare)
            print(compare, 'fidelity for', circuit_tags[alg_dex], 'with', fredkin_tags[fredkin_choice],
                    'and initial state', choice, ':', fidelity)
            comps.append(fidelity)
        fredkins[fredkin_choice].append(comps[0] - comps[1])
print('__________')

print('Calculating average differences... ')
avg3 = sum(fredkin_3) / len(fredkin_3)
avg7 = sum(fredkin_7) / len(fredkin_7)
print('Fredkin7: ', avg7)
print('Fredkin3: ', avg3)

Testing GHZ ...
__________
Ideal fidelity for GHZ with Fredkin7 and initial state 1 : 1.0
Backend fidelity for GHZ with Fredkin7 and initial state 1 : 0.734375
Ideal fidelity for GHZ with Fredkin3 and initial state 1 : 1.0
Backend fidelity for GHZ with Fredkin3 and initial state 1 : 0.8125
Ideal fidelity for GHZ with Fredkin7 and initial state 2 : 1.0
Backend fidelity for GHZ with Fredkin7 and initial state 2 : 0.685546875
Ideal fidelity for GHZ with Fredkin3 and initial state 2 : 1.0
Backend fidelity for GHZ with Fredkin3 and initial state 2 : 0.7646484375
Ideal fidelity for GHZ with Fredkin7 and initial state 3 : 1.0
Backend fidelity for GHZ with Fredkin7 and initial state 3 : 0.7099609375
Ideal fidelity for GHZ with Fredkin3 and initial state 3 : 1.0
Backend fidelity for GHZ with Fredkin3 and initial state 3 : 0.7294921875
Ideal fidelity for GHZ with Fredkin7 and initial state 4 : 1.0
Backend fidelity for GHZ with Fredkin7 and initial state 4 : 0.6953125
Ideal fidelity for GHZ with 